<a href="https://colab.research.google.com/github/samlawson1/news/blob/TENSORFLOW_MODEL/tensorflow_model_train/NYT_TensorFlow_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TensorFlow Text Classification Model**

- Training Dataset: [A Million News Headlines - Kaggle](https://www.kaggle.com/datasets/therohk/million-headlines)

1. Install dependencies [per TensorFlow docs](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).


> Google Colab is using tensorflow v 2.15.0. Will be using that instead of v 2.13.0 in the docs



In [1]:
!pip install -q tfds-nightly
!pip install -U "tensorflow-text==2.15.*"
!pip install "tf-models-official==2.15.*"


In [32]:
import os
import shutil
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from official.nlp import optimization

tf.get_logger().setLevel('ERROR')

In [18]:
#Read in dataset with pandas and show values
file = 'abcnews-date-text.csv'
df = pd.read_csv(file)
print(df.shape)
df.head()

(1244184, 2)


,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [19]:
# Randomly filter data to 100K rows for training and testing.
# I was crashing the Colab notebook training on the entire 1,244,184 row dataset

index = list(df.index)
random.shuffle(index)
print(index[0:10])

index = index[0:100000]
df = df.iloc[index]

#Reset the index to easily split into training and testing
print(df.shape)

df.head()

[297322, 1053514, 196685, 763255, 242130, 1243208, 74711, 7998, 1002970, 300875]
(100000, 2)


,publish_date,headline_text
297322,20070411,howard says climate change on coag agenda
1053514,20161019,mp paul brown absent for 10 sitting days of pa...
196685,20051026,australia urged to support trawling moratorium
763255,20130215,police unable to identify pedestrian killed by...
242130,20060609,council considers increasing sewerage rates


In [40]:
#put the headline text in a list
text = list(df['headline_text'])

#Should have 100K values
print(len(text))


#Split into train and test
train_test_split_point = 75000

train_data = text[:train_test_split_point]
test_data = text[train_test_split_point:]




100000


In [43]:
#initiate Tokenizer object
tokenizer = Tokenizer(num_words=5000,oov_token='<OOV>')
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index

#training sequences
train_sequences = tokenizer.texts_to_sequences(train_data)
train_pad = pad_sequences(train_sequences, padding='post')

#test sequences
test_sequences = tokenizer.texts_to_sequences(test_data)
test_pad = pad_sequences(test_sequences, padding = 'post')


